In [1]:
import os
import time
from datetime import datetime

import numpy as np
import pandas as pd
import scipy as sp
from tqdm import tqdm

import tensorflow as tf
import tensorflow_probability as tfp

import vegasflow
from vegasflow import VegasFlow

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
tf.test.is_gpu_available()

True

In [4]:
# tf.compat.v1.enable_eager_execution()
print(tf.config.threading.get_inter_op_parallelism_threads())
print(tf.config.threading.get_intra_op_parallelism_threads())


# tf.compat.v1.disable_eager_execution()

# tf.disable_v2_behavior()
# tf.config.threading.set_inter_op_parallelism_threads(64)
# tf.config.threading.set_intra_op_parallelism_threads(64)

0
0


In [5]:
def variance_weighted_result(means, stddevs):
    """ Computes weighted mean and stddev of given means and
        stddevs arrays, using Inverse-variance weighting
    """
    assert np.size(means) == np.size(stddevs)
    assert means.shape == stddevs.shape
    variance = 1./np.sum(1./stddevs**2, axis=-1)
    mean = np.sum(means/(stddevs**2), axis=-1)
    mean *= variance
    return mean, np.sqrt(variance)

In [6]:
NRUNS = 2

### F1 - simple gauss

In [7]:
import scipy.stats as spt

target_precision = 1e-3

norm_dist = spt.norm(loc=0.5, scale=0.05)

func_tag = 'f1'

target_dict = {
    'f1_d2': (norm_dist.cdf(1) - norm_dist.cdf(0)) ** 2,
    'f1_d4': (norm_dist.cdf(1) - norm_dist.cdf(0)) ** 4,
    'f1_d6': (norm_dist.cdf(1) - norm_dist.cdf(0)) ** 6,
    'f1_d8': (norm_dist.cdf(1) - norm_dist.cdf(0)) ** 8,
}
target_dict

{'f1_d2': 1.0, 'f1_d4': 1.0, 'f1_d6': 1.0, 'f1_d8': 1.0}

In [20]:
sigma = 0.05
alpha = sigma * np.sqrt(2)
alpha_tf = tf.constant(alpha, dtype=tf.float64)
pi = np.pi
pi_tf = tf.constant(np.pi, dtype=tf.float64)


# @tf.function(input_signature=[tf.TensorSpec(shape=(None,2), dtype=tf.float64)])
def f1_d2(x):
    print('Tracing..')
    pre = tf.cast(1.0 / (alpha * tf.sqrt(pi_tf)) ** 2, dtype=tf.float64)
    exponent = -1 * tf.reduce_sum((x - .5) ** 2, axis=-1) / alpha ** 2
    return pre * tf.exp(exponent)

def f1_d2_np(x):
    print('Tracing..')
    pre = 1.0 / (alpha * np.sqrt(pi)) ** 2
    exponent = -1 * np.sum((x - .5) ** 2, axis=-1) / alpha ** 2
    return pre * np.exp(exponent)


# @tf.function(input_signature=[tf.TensorSpec(shape=(None,4), dtype=tf.float64)])
def f1_d4(x):
    print('Tracing..')
    pre = tf.cast(1.0 / (alpha * tf.sqrt(pi_tf)) ** 4, dtype=tf.float64)
    exponent = -1 * tf.reduce_sum((x - .5) ** 2, axis=-1) / alpha ** 2
    return pre * tf.exp(exponent)


# @tf.function(input_signature=[tf.TensorSpec(shape=(None,6), dtype=tf.float64)])
def f1_d6(x):
    print('Tracing..')
    pre = tf.cast(1.0 / (alpha * tf.sqrt(pi_tf)) ** 6, dtype=tf.float64)
    exponent = -1 * tf.reduce_sum((x - .5) ** 2, axis=-1) / alpha ** 2
    return pre * tf.exp(exponent)


# @tf.function(input_signature=[tf.TensorSpec(shape=(None,8), dtype=tf.float64)])
def f1_d8(x):
    print('Tracing..')
    pre = tf.cast(1.0 / (alpha * tf.sqrt(pi_tf)) ** 8, dtype=tf.float64)
    exponent = -1 * tf.reduce_sum((x - .5) ** 2, axis=-1) / alpha ** 2
    return pre * tf.exp(exponent)

In [9]:
dim2func_dict = {
    2: f1_d2,
    4: f1_d4,
    6: f1_d6,
    8: f1_d8,
}
dim2func_dict

{2: <function __main__.f1_d2(x)>,
 4: <function __main__.f1_d4(x)>,
 6: <function __main__.f1_d6(x)>,
 8: <function __main__.f1_d8(x)>}

In [10]:
# with tf.compat.v1.Session() as sess:
    
#     # print(f1_d8(tf.random.uniform(shape=(10,8), dtype=tf.float64)))
#     print(sess.run(f1_d8(tf.random.uniform(shape=(10,8), dtype=tf.float64))))

In [11]:
# from vegasflow import run_eager

# run_eager(False)

In [23]:
dims = 8
n_calls = int(1e8)
vegas_instance = VegasFlow(dims, n_calls, verbose=1,  list_devices=['CPU'])

vegas_instance.compile(f1_d8, compilable=True)

In [24]:
n_iter = 1
result = vegas_instance.run_integration(n_iter)

Tracing..nt to the computing device: 1.0 %


[INFO] (vegasflow.monte_carlo) Result for iteration 0: 0.8338 +/- 0.0697(took 110.70489 s)
[INFO] (vegasflow.monte_carlo)  > Final results: 0.8338 +/- 0.0697128


tf.Tensor(205.76677645409094, shape=(), dtype=float64)


In [25]:
result

(0.8337999424519433, 0.06971277537357445)

In [ ]:
# %%time
# np.random.seed(123)

# result_means = []
# result_sdevs = []
# result_times = []
# result_sum_nevals = []
# result_run_nums = []

# ndims_lst = [2,4,6,8]
# neval_lst = list(map(int, [1e5, 1e6]))  # 1e8
# nitn_lst = list(map(int, [2,4]))
# run_lst = list(range(1,NRUNS+1))

# for run in run_lst:
#     print(f'run={run}')
#     for ndims in ndims_lst:
#         integrand = dim2func_dict[ndims]
        
#         for nitn in nitn_lst:
#             for neval in neval_lst:

#                 print(f'ndims={ndims}  nitn={nitn}  neval={neval}')

#                 integ = vegas.Integrator([[0, 1]] * ndims)

#                 time_a = time.time()
#                 current_result = integ(integrand, nitn=nitn, neval=neval)
#                 current_result_mean = current_result.mean
#                 current_result_sdev = current_result.sdev
#                 total_time = time.time() - time_a

#                 result_means.append(current_result_mean)
#                 result_sdevs.append(current_result_sdev)
#                 result_times.append(total_time)
#                 result_sum_nevals.append(current_result.sum_neval)
#                 result_run_nums.append(run)

#                 # print(current_result_mean, current_result_stddev)